In [ ]:
#
# data cleaning/preprocessing
#

import scipy as sp
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

def data_preprocessing(data, label):
    # regularization 표준화
    regularization = StandardScaler()
    regularization.fit(data)
    data_scaled = regularization.transform(data)
    data_scaled = pd.DataFrame(data=data_scaled, columns=data.columns)
    data_scaled[data_scaled.columns[0]] = label
    
    # check Z score
    zscore = pd.DataFrame()
    outlier = {}
    outlier_idx = []
    for col in data_scaled.columns:
        if col != data_scaled.columns[0]:
            zscore[f'{col}_z'] = sp.stats.zscore(data_scaled[col])
            outlier[col] = zscore[f'{col}_z'][(zscore[f'{col}_z']>2.58) | (zscore[f'{col}_z']<-2.58)] # 99% = 2.58
            outlier_idx.append(list(outlier[col].index))    
    outlier_idx_sum = sum(outlier_idx,[])
    data_scaled = data_scaled.drop(outlier_idx_sum)

    # IQR outlier + mean으로 대체
    data_pre = data
    for col in data_pre.columns:
        if col != data_pre.columns[0]:
            q1 = np.quantile(data_pre[f'{col}'], 0.25)
            q3 = np.quantile(data_pre[f'{col}'], 0.75)
            IQR = q3 - q1
            condition = (data_pre[f'{col}'] < (q1 - 1.5 * IQR)) | (data_pre[f'{col}'] > (q3 + 1.5 * IQR))
            outlier = data_pre[condition]
            data_pre[f'{col}'] = data_pre[f'{col}'].replace([data_pre[f'{col}'][outlier.index]], data_pre[f'{col}'].mean())            

    # Normalization
    normalization = MinMaxScaler()
    normalization.fit(data_pre)
    data_nor = normalization.transform(data_pre)
    data_nor = pd.DataFrame(data=data_nor, columns=data_pre.columns)
    data_nor[data_nor.columns[0]] = data_pre[data_pre.columns[0]]
    data_nor[data_nor.columns[0]] = label
    
    return data_pre 